In [103]:
import duckdb
import pandas as pd

In [104]:
con = duckdb.connect("ontario_rentals.duckdb")

### Query 1 - Dataset Integrity Check

**Purpose:**
- Confirm deduplication
- Validate price cleaning
- Measure low-price marketing cases

In [105]:
query_1 = """
SELECT
    COUNT(*) AS total_rows,
    COUNT(DISTINCT url) AS unique_urls,
    SUM(CASE WHEN extreme_low_price_flag = 1 THEN 1 ELSE 0 END) AS low_price_flag_count,
    SUM(CASE WHEN price_monthly IS NULL THEN 1 ELSE 0 END) AS null_price_count
FROM kijiji_rentals_clean;
"""

df_q1 = con.execute(query_1).fetchdf()
df_q1


,total_rows,unique_urls,low_price_flag_count,null_price_count
0,4040,4040,13.0,153.0


**Interpretation:**
- Total rows reflect the fully cleaned and deduplicated dataset.
- 'unique_urls' confirm successful duplicate removal at the pipeline level.
- 'low_price_flag_count' captures extreme low-price marketing cases (e.g., $1 listings, parking rentals).
- 'null_price_count reflects listings where price is unavailable or "Please Contact".
- This confirms the cleaning logic is fucntional as intended and the dataset is suitable for structured analysis.

### Query 2 - Listing Volume by Market Area

**Purpose:**
- Measure listing distribution across GTA market areas
- Identify supply concentration
- Provide context for rent comparisons

In [106]:
query_2 = """
SELECT
  market_area,
  COUNT(*) AS listings,
  ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS pct_share
FROM kijiji_rentals_clean
GROUP BY market_area
ORDER BY listings DESC;
"""

df_q2 = con.execute(query_2).fetchdf()
df_q2

,market_area,listings,pct_share
0,Toronto,2303,57.00
1,Peel,940,23.27
2,Halton,298,7.38
3,Durham,224,5.54
4,York,208,5.15
5,Other / Unknown,67,1.66


**Interpretation:**
- The majority of listings are concentrated in Toronto region (57%).
- Secondary markets include Peel Region (23%), Halton Region (8%), and Durham Region (6%).
- This confirms supply imbalance across GTA subregions.

### Query 3 - Price Distibution by Market Area (median, quartiles)

**Purpose:**
- Compare rent levels across GTA market areas.
- Use robust stats (median + quartiles) so outliers do not dominate.
- Restrict to reasonable residential records (exclude flagged extreme-low cases).

In [107]:
query_3 = """
SELECT
  market_area,
  COUNT(*) AS listings,
  ROUND(AVG(price_monthly), 0) AS avg_price,
  QUANTILE_CONT(price_monthly, 0.25) AS p25_price,
  QUANTILE_CONT(price_monthly, 0.50) AS median_price,
  QUANTILE_CONT(price_monthly, 0.75) AS p75_price,
  MIN(price_monthly) AS min_price,
  MAX(price_monthly) AS max_price
FROM kijiji_rentals_clean
WHERE price_monthly IS NOT NULL
  AND extreme_low_price_flag = 0
GROUP BY market_area
HAVING COUNT(*) >= 30
ORDER BY median_price DESC;
"""

df_q3 = con.execute(query_3).fetchdf()
df_q3

,market_area,listings,avg_price,p25_price,median_price,p75_price,min_price,max_price
0,York,199,2501.0,1825.0,2350.0,2897.50,850,10000
1,Halton,292,2655.0,1999.0,2350.0,2959.25,600,11995
2,Toronto,2257,2414.0,1875.0,2250.0,2700.00,400,13500
3,Peel,848,2250.0,1698.0,2199.0,2750.00,400,22500
4,Durham,219,2209.0,1725.0,2100.0,2625.00,700,5500
5,Other / Unknown,59,2047.0,1727.5,2000.0,2375.00,325,3300


**Interpretation:**
- Areas with the highest median rents are: York, Halton, and Toronto.
- The IQR (p75 − p25) highlights variability; the widest IQR suggests more mixed inventory (e.g., condos + houses + basement units).
- Minimum prices should be sanity-checked for remaining anomalies; maximum prices likely reflect luxury or multi-bedroom listings.

### Query 4 - Median rent by bedrooms and market_area

**Purpose**
- Understand whether higher median rents in York and Halton are driven by larger unit sizes.
- Compare like-for-like units (e.g., 1BR vs 1BR across regions).

In [112]:
query_4 = """
SELECT
  market_area,
  bedrooms,
  COUNT(*) AS listings,
  QUANTILE_CONT(price_monthly, 0.50) AS median_price
FROM kijiji_rentals_clean
WHERE price_monthly IS NOT NULL
  AND extreme_low_price_flag = 0
  AND bedrooms IS NOT NULL
GROUP BY market_area, bedrooms
HAVING COUNT(*) >= 20
ORDER BY market_area, bedrooms;
"""

df_q4 = con.execute(query_4).df()
df_q4

,market_area,bedrooms,listings,median_price
0,Durham,1.0,49,1600.0
1,Durham,2.0,82,2000.0
2,Durham,3.0,52,2600.0
3,Halton,1.0,59,1945.0
4,Halton,2.0,133,2300.0
5,Halton,3.0,45,3000.0
6,Halton,4.0,20,3700.0
7,Other / Unknown,2.0,20,2000.0
8,Peel,1.0,205,1650.0
9,Peel,1.5,36,2070.0


**Interpretation:** 

Key Observations:
- Rent increases nonlinearly as bedroom count rises across all regions.
- Toronto commands the highest premiums for 2–3 bedroom units relative to outer regions.
- Durham consistently shows the lowest median rents across comparable bedroom counts.
- 1.5 and 2.5 bedroom categories appear and likely reflect den-based or flex layouts common in condos.
- The price jump from 2 → 3 bedrooms is materially larger than from 1 → 2 in most regions, suggesting stronger demand for family-sized units.

Market Implications:
- Toronto shows strong vertical price scaling by unit size.
- Halton and York display premium pricing for 3+ bedroom inventory, likely reflecting suburban detached housing.
- Peel shows competitive pricing in 2-bedroom units, potentially indicating higher supply concentration.

Data Considerations:
- 0-bedroom units (studios) in Toronto may include some single room listings.
- Median pricing reduces distortion from luxury outliers (e.g., $10k+ listings).

### Query 5 - Median Rent by Unit Type

**Purpose:**
- Compare pricing structures across apartments, condos, houses, and basements
- Identify structural pricing differences by property type
- Detect whether certain unit types drive price dispersion

In [114]:
query_5 = """
SELECT
    unit_type_clean,
    COUNT(*) AS listings,
    AVG(price_monthly) AS avg_price,
    MEDIAN(price_monthly) AS median_price,
    MIN(price_monthly) AS min_price,
    MAX(price_monthly) AS max_price
FROM kijiji_rentals_clean
WHERE price_monthly IS NOT NULL
  AND extreme_low_price_flag = 0
GROUP BY unit_type_clean
ORDER BY median_price DESC;
"""

df_q5 = con.execute(query_5).fetchdf()
df_q5

,unit_type_clean,listings,avg_price,median_price,min_price,max_price
0,Townhouse,205,2833.214634,2850.0,400,7300
1,House,403,2616.044665,2700.0,400,12499
2,NaN,864,2928.946759,2590.0,875,22500
3,Condo,555,2432.311712,2350.0,750,7800
4,Duplex/Triplex,42,2126.880952,2200.0,700,3200
5,Apartment,1075,2158.861395,2099.0,325,5500
6,Basement,724,1796.954420,1750.0,400,6800
7,Room,6,1183.333333,1100.0,950,1800


**Interpretation:**
- Townhouses command the highest median rent (~$2,850), followed by Houses (~$2,700), indicating a premium for larger, family-oriented property types.
- Condo median rent (~$2,350) sits meaningfully above Apartments (~$2,099), suggesting stronger pricing power for condo units, likely reflecting amenities and newer builds.
- Basements and rooms remain the most affordable segment (~$1,750, and ~$1,100 median, respectively), reinforcing their role as entry-level rental options in the GTA market.
- The “Unclassified” (NaN) segment has a high average (~$2,929) and wide price range (max ~$22,500), suggesting it likely contains luxury listings and premium units not captured by dropdown categories.
- Overall, pricing follows a clear structural hierarchy:
  **Townhouse > House > Condo > Duplex/Triplex > Apartment > Basement > Room**, consistent with expected space and amenity premiums.

### Query 6 - Price per Square Foot by Market Area
**Purpose:**
- Standardize rental pricing by calculating price per square foot to enable size-adjusted comparisons.
- Identify which market areas command true pricing premiums independent of unit size.

In [118]:
query_6 = """
SELECT
    market_area,
    COUNT(*) AS listings,
    ROUND(AVG(price_monthly / sqft), 2) AS avg_price_per_sqft,
    MEDIAN(price_monthly / sqft) AS median_price_per_sqft
FROM kijiji_rentals_clean
WHERE price_monthly IS NOT NULL
  AND sqft IS NOT NULL
  AND sqft > 0
  AND extreme_low_price_flag = 0
GROUP BY market_area
ORDER BY median_price_per_sqft DESC;
"""

df_q6 = con.execute(query_6).fetchdf()
df_q6

,market_area,listings,avg_price_per_sqft,median_price_per_sqft
0,Toronto,1068,4.35,2.932415
1,Halton,101,2.58,2.496200
2,Other / Unknown,38,2.51,2.412980
3,York,71,8.05,2.228571
4,Durham,95,2.38,2.142857
5,Peel,491,2.68,2.083333


**Interpretation:**
- Toronto commands the highest size-adjusted premium, with the highest median price per sqft (~2.93), confirming that core urban demand remains strong even after controlling for unit size.
- York shows a large gap between average (8.05) and median (2.23) price per sqft, indicating the presence of extreme high-end outliers inflating the mean; median is the more reliable central tendency here.
- Peel and Durham exhibit the lowest median price per sqft (~2.08 – 2.14), suggesting relative affordability once unit size is standardized.
- The dispersion between average and median price per sqft across markets highlights luxury skew and heterogeneous inventory composition, reinforcing the importance of using medians in rental analytics.


### Query 7 - Inventory Mix by Unit Type and Market Area
**Purpose:**
- Analyze the distribution of housing types within each market area.
- Identify structural differences in housing supply across regions.
- Provide context for observed pricing differences by revealing composition effects (e.g., condo-heavy vs house-heavy markets).

In [122]:
query_7 = """
SELECT
    market_area,
    unit_type_clean,
    COUNT(*) AS listings,
    ROUND(100.0 * COUNT(*) 
          / SUM(COUNT(*)) OVER (PARTITION BY market_area), 2) AS pct_within_market
FROM kijiji_rentals_clean
WHERE unit_type_clean IS NOT NULL
GROUP BY market_area, unit_type_clean
ORDER BY market_area, listings DESC;
"""

df_q7 = con.execute(query_7).fetchdf()
df_q7

,market_area,unit_type_clean,listings,pct_within_market
0,Durham,Basement,71,38.38
1,Durham,Apartment,43,23.24
2,Durham,House,25,13.51
3,Durham,Townhouse,19,10.27
4,Durham,Condo,15,8.11
5,Durham,Duplex/Triplex,12,6.49
6,Halton,Apartment,79,35.91
7,Halton,Condo,44,20.00
8,Halton,Basement,42,19.09
9,Halton,Townhouse,29,13.18


**Interpretation:**
- Toronto is apartment-dominant (44.9%), followed by condos (21.5%) and basements (17.1%), reflecting a high-density, multi-unit urban rental structure.
- Peel and Durham are basement-heavy markets (~40%), suggesting strong secondary-suite supply and a more affordability-driven rental mix relative to Toronto.
- York differs structurally, with basements (34.4%) and condos (27.0%) leading inventory, while apartments represent less than 8% of listings.
- Detached houses remain a minority share across regions (≈10–22%), indicating that subdivided or multi-unit housing forms dominate rental supply.
- Duplex/Triplex inventory is negligible (<2% in most markets), highlighting limited mid-density “missing middle” rental stock.


### Query 8 - Bedroom Premium Curve (Marginal Rent by Bedroom Count)

**Purpose:**
- Analyze how median rent changes as bedroom count increases.  
- Quantify the incremental pricing premium associated with each additional bedroom.  
- Evaluate whether rental pricing scales proportionally with unit size or exhibits diminishing marginal returns.


In [125]:
query_8 = """
SELECT
    bedrooms,
    COUNT(*) AS listings,
    QUANTILE_CONT(price_monthly, 0.50) AS median_price
FROM kijiji_rentals_clean
WHERE price_monthly IS NOT NULL
  AND extreme_low_price_flag = 0
  AND bedrooms IS NOT NULL
GROUP BY bedrooms
ORDER BY bedrooms;
"""

df_q8 = con.execute(query_8).fetchdf()
df_q8

,bedrooms,listings,median_price
0,0.0,208,1645.0
1,1.0,1148,1895.0
2,1.5,126,2100.0
3,2.0,1350,2349.0
4,2.5,57,2300.0
5,3.0,659,2895.0
6,3.5,31,2800.0
7,4.0,200,3300.0
8,4.5,18,3300.0
9,5.0,74,4500.0


**Interpretation:**
- Median rent increases consistently with bedroom count, confirming a clear positive bedroom premium (e.g., $1,645 for studios vs. $4,500 for 5-bedroom units).
- The incremental premium is not linear; rent jumps are larger between 2 → 3 and 4 → 5 bedrooms, indicating higher marginal value for larger family-sized units.
- Fractional bedroom categories (1.5, 2.5, 3.5, 4.5) show smaller listing counts and slight pricing irregularities, suggesting niche inventory segments rather than mainstream supply.

In [127]:
create_sample_csv = """
COPY (
    SELECT *
    FROM kijiji_rentals_clean
    WHERE price_monthly IS NOT NULL
    QUALIFY ROW_NUMBER() OVER (PARTITION BY market_area ORDER BY RANDOM()) <= 50
)
TO 'kijiji_rentals_clean_sample.csv'
WITH (HEADER, DELIMITER ',');
"""

df_csv = con.execute(create_sample_csv).fetchdf()
df_csv

,Count
0,300
